In [1]:
%load_ext autoreload
%autoreload 2

import os
ROOT_DIR = os.getcwd()[:os.getcwd().rfind('NVcenter')]+ 'NVcenter'
os.chdir(ROOT_DIR)

In [9]:
from NVcenter import *
import numpy as np

## Demos

In [30]:
# Spin class

nv1_config = ('NV', (0,0,0), 1, {})
nv1 = Spin(*nv1_config)
nv1.H

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 2.4571932e+09]]

In [32]:
# TwoSpinSystem class

nv1_config = ('NV', (0,0,0), 1, {})
nv2_config = ('NV', (20e-9,0,0), 1, {})

two_spin_system = TwoSpinSystem(nv1_config, nv2_config, 100e-6)
two_spin_system.H

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.12495312e+04]
 [ 0.00000000e+00  2.45719320e+09 -2.04165104e+04  0.00000000e+00]
 [ 0.00000000e+00 -2.04165104e+04  2.45719320e+09  0.00000000e+00]
 [-6.12495312e+04  0.00000000e+00  0.00000000e+00  4.91442724e+09]]

In [38]:
# SpinConfig class

register_C13_pos = [(0.87e-9, 0, 0.19e-9)]
bath_spin_type = 'C13'
abundancy, rmin, rmax = 0.02e-2, 1.8e-9, 4.2e-9
spin_config = SpinConfig(register_C13_pos, bath_spin_type, abundancy, rmin, rmax, 
                     bath_spin_pos_seed=10, bath_init_state_idx=123, bath_P1_seed=123)

bath_approx_level = 'no_bath'
spin_config.mf_configs[bath_approx_level][0]
spin_config.system_configs[bath_approx_level][0]

('NV', (0, 0, 0), 0, {})

In [40]:
# System class

register_C13_pos = [(0.87e-9, 0, 0.19e-9)]
bath_spin_type = 'C13'
abundancy, rmin, rmax = 0.02e-2, 1.8e-9, 4.2e-9
spin_config = SpinConfig(register_C13_pos, bath_spin_type, abundancy, rmin, rmax, 
                     bath_spin_pos_seed=10, bath_init_state_idx=123, bath_P1_seed=123)

bath_approx_level = 'no_bath'
system_configs, mf_configs = spin_config.system_configs[bath_approx_level], spin_config.mf_configs[bath_approx_level]

system = System(system_configs, mf_configs)
system.H

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-7.92420875e+04  0.00000000e+00  3.90984042e+04  1.79029535e+05]
 [ 0.00000000e+00  7.92420875e+04  5.39840237e+04 -3.90984042e+04]
 [ 3.90984042e+04  5.39840237e+04  2.45719031e+09 -5.52934935e+04]
 [ 1.79029535e+05 -3.90984042e+04 -5.52934935e+04  2.45719610e+09]]

## Suter

In [44]:
import qutip as q

D = DEFAULTS['D_gs'] # 2.87e9
ve = DEFAULTS['gamma_e'] * DEFAULTS['Bz']/ (2*np.pi) #  -414e6
vc = DEFAULTS['gamma_C'] * DEFAULTS['Bz']/ (2*np.pi) # 0.158e6
A_N = -2.16e6
A_zz = -0.152e6
A_zx = 0.110e6
I_NV, Sx_NV, Sy_NV, Sz_NV = get_spin_matrices(1, trunc=True)
I_C13, Sx_C13, Sy_C13, Sz_C13 = get_spin_matrices(1/2, trunc=False)

H = D * adjust_space_dim(2, Sz_NV**2, 0) - (ve - A_N) * adjust_space_dim(2, Sz_NV, 0) - vc * adjust_space_dim(2, Sz_C13, 1) + A_zz * q.tensor(Sz_NV, Sz_C13) + A_zx * q.tensor(Sz_NV, Sx_C13)
H

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-7.92420875e+04  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.92420875e+04  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  2.45758996e+09 -5.50000000e+04]
 [ 0.00000000e+00  0.00000000e+00 -5.50000000e+04  2.45759645e+09]]

In [50]:
# we can ignore the upper left and lower right blocks because they connect states with an spin flip in the NV center. 
# But this will not happen because of the big energy level splitting.

config_nv = ('NV', (0,0,0), 1, {})
config_C13 = ('C13', (0.87e-9, 0, 0.19e-9), 0, {})

two_spin_config = TwoSpinSystem(config_nv, config_C13, 100e-6)
two_spin_config.H

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[-7.92420875e+04  0.00000000e+00  3.90984042e+04  1.79029535e+05]
 [ 0.00000000e+00  7.92420875e+04  5.39840237e+04 -3.90984042e+04]
 [ 3.90984042e+04  5.39840237e+04  2.45719031e+09 -5.52934935e+04]
 [ 1.79029535e+05 -3.90984042e+04 -5.52934935e+04  2.45719610e+09]]

In [51]:
two_spin_config.dipolar_matrix

array([[ 329530.93503144,      -0.        ,  110586.98699471],
       [     -0.        , -176841.05804962,      -0.        ],
       [ 110586.98699471,      -0.        , -152689.87698181]])

## Ramsey Two NV centers

In [10]:
nv1_config = ('NV', (0,0,0), 1, {})
nv2_config = ('NV', (20e-9,0,0), 1, {})

nv1 = Spin(*nv1_config)
nv2 = Spin(*nv2_config)

In [28]:
register = TwoSpinSystem(nv1_config, nv2_config, 100e-6)

In [ ]:
register_C13_pos = [(0.87e-9, 0, 0.19e-9)]
bath_spin_type = 'C13'
abundancy, rmin, rmax = 0.02e-2, 1.8e-9, 4.2e-9
spin_config = SpinConfig(register_C13_pos, bath_spin_type, abundancy, rmin, rmax, 
                     bath_spin_pos_seed=10, bath_init_state_idx=123, bath_P1_seed=123)

In [ ]:
Pulses()